# Collecting Dataset Code 

(10/27/2022) not final virsion, it will be modified later 


## Import

In [1]:
import cv2
import os
import math
import numpy as np
import pandas as pd
import mediapipe as mp

**How to use Medaipipe (details about mediapipe attributes)**

mp.solutions.hands
- max_num_hands: the maximum number of hand will be detected by mediapipe in a single frame 
- min_detection_confidence: Minimum confidence value (between 0 and 1) for the hand detection to be considered successful
- min_tracking_confidence: Minimum confidence value (between 0 and 1) for the hand landmarks to be considered tracked successfully

mediapipe method applicable to output(result) 
- MULTI_HAND_LANDMARKS: Collection of detected/tracked hands, where each hand is represented as a list of 21 hand landmarks and each landmark is composed of x, y and z
- MULTI_HANDEDNESS: Collection of handedness of the detected/tracked hands (i.e. is it a left or right hand). Each hand is composed of label and score
    - label is a string of value either "Left" or "Right"
    
 
**Which types of 3D coordinates system can we use**

3D Coordinate System 
- origianl
    - MULTI_HAND_LANDMARKS: normalized by image's width and height, z is wrist beding the origin 
    - MULTI_HAND_WORLD_LANDMARKS: real-world corrdinates in meters 
- relative
- normalize 

In [2]:
# Setting for Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1,              # Only two hand detected 
                       min_detection_confidence=0.7) # Defualt 0.5

In [2]:
# Functions for bounding rectangle
def calc_bounding_rect(image, landmarks): # Calculate bounding rectangle 
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]
        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)
    
    return [x, y, x + w, y + h]

def draw_bounding_rect(use_brect, image, brect): # Draw bounding rectangle 
    if use_brect:
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (0, 0, 0), 1)

    return image

## Start Collecting Datasets

To collect dataset, just run below cell

- to capture and make data, press '1'
- to quit the code, press 'q' 

In [23]:
# Initialize for datasets 
path = os.getcwd()
img_cnt = 1
landmarks = []
df = pd.DataFrame()

# Initialize the webcam 
video_capture = cv2.VideoCapture(0)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 300)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 300)

while True:
    # Read each frame from the webcam
    _, frame = video_capture.read()
    x, y, c = frame.shape
    
    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    result = hands.process(framergb)
    if result.multi_hand_landmarks:
        for handslms, handness in zip(result.multi_hand_landmarks, # Be carefule to set attribute for right coordinate system
                                      result.multi_handedness): 
            landmarks.clear() # For empty list 
            for point in mp_hands.HandLandmark: # 0 ~ 20
                x = handslms.landmark[point].x
                y = handslms.landmark[point].y
                z = handslms.landmark[point].z
                landmarks.append([str(point), handness.classification[0].label, x, y, z])
            
            # Drawing landmarks on frames with bounding rectangle
            brect = calc_bounding_rect(frame, handslms)
            frame = draw_bounding_rect(True, frame, brect)
            mp_drawing.draw_landmarks(frame, handslms, mp_hands.HAND_CONNECTIONS)
    
    # To capture the frame 
    if cv2.waitKey(1) == ord('1') :
        img_name = path+"\\Dataset\\Dataset03\\id03_frame_{}.png".format(img_cnt)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name)) # If capture is successful
        img_cnt += 1
        df = df.append(pd.DataFrame(landmarks), ignore_index=True)
    
    # To Quit from application, press "q"
    elif cv2.waitKey(1) == ord('q'): 
        img_cnt -= 1
        break
        
    # Show the final output
    cv2.imshow("Output", frame)
    
# Release the webcam and destroy all active windows
print("{} images captured!".format(img_cnt)) # Print how many images captured
video_capture.release()
cv2.destroyAllWindows()

C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_1.png written!


C:\Users\mtang\AppData\Local\Temp\ipykernel_23448\1571656959.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(landmarks), ignore_index=True)


C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_2.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_3.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_4.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_5.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_6.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_7.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_8.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_9.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_10.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_11.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\i

C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_90.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_91.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_92.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_93.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_94.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_95.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_96.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_97.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_98.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_99.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dat

C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_177.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_178.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_179.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_180.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_181.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_182.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_183.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_184.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_185.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_186.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\D

C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_265.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_266.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_267.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_268.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_269.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_270.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_271.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_272.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_273.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_274.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\D

C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_353.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_354.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_355.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_356.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_357.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_358.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_359.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_360.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_361.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\Dataset\Dataset03\id03_frame_362.png written!
C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin\D

In [24]:
# Function for preprocessing dataset, to make relative landmark from the first point "wrist"
def create_relative_landmark(img_cnt, landmark_df) :
    df = pd.DataFrame()
    for i in range(img_cnt) : 
        base_lm = landmark_df.iloc[i*21, 2:].values
        for j in range(21) :
            target_lm = landmark_df.iloc[i*21+j, 2:].values 
            result_lm = target_lm-base_lm
            df = df.append(pd.DataFrame(result_lm).transpose(),  ignore_index=True)
    return df
        
relative_df = create_relative_landmark(img_cnt, df)
result_df = pd.concat([df, relative_df], axis=1)
result_df

C:\Users\mtang\AppData\Local\Temp\ipykernel_23448\1589299579.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(result_lm).transpose(),  ignore_index=True)


,0,1,2,3,4,0,1,2
0,HandLandmark.WRIST,Right,0.365295,0.833528,-4.024409e-07,0.0,0.0,0.0
1,HandLandmark.THUMB_CMC,Right,0.444711,0.771690,1.127641e-02,0.079416,-0.061838,0.011277
2,HandLandmark.THUMB_MCP,Right,0.500004,0.678157,7.499660e-03,0.13471,-0.155371,0.0075
3,HandLandmark.THUMB_IP,Right,0.531983,0.595765,1.806879e-03,0.166688,-0.237763,0.001807
4,HandLandmark.THUMB_TIP,Right,0.578436,0.549514,-8.633840e-03,0.213142,-0.284014,-0.008633
...,...,...,...,...,...,...,...,...
8395,HandLandmark.RING_FINGER_TIP,Left,0.688293,0.644720,-1.041960e-01,0.057632,-0.244637,-0.104196
8396,HandLandmark.PINKY_MCP,Left,0.720804,0.673623,-1.234965e-01,0.090143,-0.215734,-0.123497
8397,HandLandmark.PINKY_PIP,Left,0.746430,0.539594,-1.530507e-01,0.115769,-0.349763,-0.153051
8398,HandLandmark.PINKY_DIP,Left,0.770282,0.478603,-1.399251e-01,0.139621,-0.410754,-0.139925


## Export Datasets

In [25]:
from pathlib import Path  

# You need to change root for dataset!
filepath = path+"\\Dataset\\Dataset03\\"

# You need to change the csv file name! 
result_df.to_csv(filepath+"gesture03.csv", mode="w") 